# Mandatory data

In [140]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown
import geopandas
import plotly.express as px
import numpy

## Editing the table
Doing some basic work to make tha table more readable

In [ ]:
url = "../Data/Annual_population_eurostat(mandatory)_raw.tsv"
table_raw = pd.read_csv(url,sep='\t')
print(table_raw.columns)

table_raw.rename(columns={"freq,citizen,age,unit,sex,geo\TIME_PERIOD":"title"},inplace=True)
display(table_raw.head(5))


Index(['freq,citizen,age,unit,sex,geo\TIME_PERIOD', '2015 ', '2016 ', '2017 ',
       '2018 ', '2019 ', '2020 ', '2021 ', '2022 ', '2023 ', '2024 '],
      dtype='object')


<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_54708/2102777931.py:6: SyntaxWarning: invalid escape sequence '\T'
  table_raw.rename(columns={"freq,citizen,age,unit,sex,geo\TIME_PERIOD":"title"},inplace=True)


,title,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,"A,EU28_FOR,TOTAL,NR,F,AT",288028,310759,329981,348012,365125,383295,:,:,:,:
1,"A,EU28_FOR,TOTAL,NR,F,BE",416232,428008,434889,440355,448655,457258,:,:,:,:
2,"A,EU28_FOR,TOTAL,NR,F,CH",594052,610566,622893,630371,637330,645916,:,:,:,:
3,"A,EU28_FOR,TOTAL,NR,F,CY",55730,57716,57673,59787,63044,62935,:,:,:,:
4,"A,EU28_FOR,TOTAL,NR,F,CZ",72465,76736,81670,86132,91296,96186,:,:,:,:


In [142]:
citizens = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row["title"].split(","))
    # print(title_split)
    citizens.append(title_split[1]) 
    sex.append(title_split[4])
    country.append(title_split[5])
    
    

table_raw["citizen"] = citizens
table_raw["sex"]=sex
table_raw["country"]=country

table_raw.drop(columns=["title"],inplace=True)
table_raw.set_index("country",inplace=True)



#### Reindexovanie


In [143]:
print(table_raw.columns)
new_order = ['citizen', 'sex','2015 ', '2016 ', '2017 ', '2018 ', '2019 ', '2020 ', '2021 ', '2022 ',
       '2023 ', '2024 ']
table = table_raw[new_order].reset_index()

display(table.head())

Index(['2015 ', '2016 ', '2017 ', '2018 ', '2019 ', '2020 ', '2021 ', '2022 ',
       '2023 ', '2024 ', 'citizen', 'sex'],
      dtype='object')


,country,citizen,sex,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,AT,EU28_FOR,F,288028,310759,329981,348012,365125,383295,:,:,:,:
1,BE,EU28_FOR,F,416232,428008,434889,440355,448655,457258,:,:,:,:
2,CH,EU28_FOR,F,594052,610566,622893,630371,637330,645916,:,:,:,:
3,CY,EU28_FOR,F,55730,57716,57673,59787,63044,62935,:,:,:,:
4,CZ,EU28_FOR,F,72465,76736,81670,86132,91296,96186,:,:,:,:


#### Úprava hodnôt

In [144]:
# display(
#     table_raw[table_raw['sex'] == 'T']
#     .dropna(subset=['2024 ']) 
#     .sort_values(by="2024 ", ascending=False)
#     .head(200)
# )


print(table.iloc[1,10])

replaced = table.replace(to_replace=table.iloc[1,10],value=numpy.nan)

display(replaced.head())


: 


,country,citizen,sex,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,AT,EU28_FOR,F,288028,310759,329981,348012,365125,383295,NaN,NaN,NaN,NaN
1,BE,EU28_FOR,F,416232,428008,434889,440355,448655,457258,NaN,NaN,NaN,NaN
2,CH,EU28_FOR,F,594052,610566,622893,630371,637330,645916,NaN,NaN,NaN,NaN
3,CY,EU28_FOR,F,55730,57716,57673,59787,63044,62935,NaN,NaN,NaN,NaN
4,CZ,EU28_FOR,F,72465,76736,81670,86132,91296,96186,NaN,NaN,NaN,NaN


#### Uloženie do súboru

In [145]:
path = "../Data/Annual_population_eurostat(mandatory).csv"

replaced.to_csv(path,index=False)
